In [1]:
import math
import re
import pandas as pd
import numpy as np
import pymorphy2
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
import nltk
from nltk.tokenize import WhitespaceTokenizer
from string import punctuation
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
#from tqdm import tqdm
from scipy.sparse import hstack
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, roc_curve
#import xgboost as xgb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from keras.utils.np_utils import to_categorical

%matplotlib inline

Using TensorFlow backend.


In [ ]:
df_init = pd.read_csv('X_train.csv').drop(['userName', 'property', 'date'], axis=1)

In [ ]:
def merge_com(data):
    data = data.copy()
    for i in range(data.shape[0]):
        data.loc[i, 'com_len'] = len(data.loc[i, 'comment'])
        
        try:
            data.loc[i, 'comPos_len'] = len(data.loc[i, 'commentPositive'])
        except TypeError:
            data.loc[i, 'comPos_len'] = 0
            
        try:
            data.loc[i, 'comNeg_len'] = len(data.loc[i, 'commentNegative'])
        except TypeError:
            data.loc[i, 'comNeg_len'] = 0
            
        data.loc[i, 'nCapital'] = len(re.findall("[A-Z]", data.loc[i, 'comment']))
        
        try:
            data.loc[i, 'nCapitalPos'] = len(re.findall("[A-Z]", data.loc[i, 'commentPositive']))
        except TypeError:
            data.loc[i, 'nCapitalPos'] = 0
            
        try:
            data.loc[i, 'nCapitalNeg'] = len(re.findall("[A-Z]", data.loc[i, 'commentNegative']))
        except TypeError:
            data.loc[i, 'nCapitalNeg'] = 0
        
        try:
            if math.isnan(data.commentNegative[i]):
                data.loc[i, 'hasNegComment'] = 0
        except TypeError:
            data.loc[i, 'comment'] = data.loc[i, 'comment'] + ' ' + data.loc[i, 'commentNegative']
            data.loc[i, 'hasNegComment'] = 1
        
        try:
            if math.isnan(data.commentPositive[i]):
                data.loc[i, 'hasPosComment'] = 0
        except TypeError:
            data.loc[i, 'comment'] = data.comment[i] + ' ' + data.commentPositive[i]
            data.loc[i, 'hasPosComment'] = 1
        
        data.loc[i, 'n_of_exc'] = data.loc[i, 'comment'].count('!')
        # if i%1000==0:
        #     print(i)
    return data.drop(['commentNegative', 'commentPositive'], axis=1)

In [ ]:
%%time
df_merged_comments = merge_com(df_init)

In [ ]:
morph = MorphAnalyzer()
mystopwords = list(set(stopwords.words('russian')) - set(['много','без','никогда' , 'совсем' , 'не', 'нет', 'более','больше',  'ничего', "но", "хорошо", "лучше"])) + ['это',"х","р", "тыс", "тыщ", "руб"] 

def parse_sentence(sent):
    sent = sent.replace('...', ' ')
    sent = sent.replace('-', ' ')
    sent = sent.replace('/', ' ')
    exclude = set(punctuation + '0123456789'+u'–—'+u'«»')
    merged_text = ''.join(ch for ch in sent if ch not in exclude)
    tokens = WhitespaceTokenizer().tokenize(merged_text.lower())
    tokens = [t_ for t_ in tokens if t_ not in mystopwords]
    tokens = [morph.parse(word)[0].normal_form for word in tokens]
    return ' '.join(ch for ch in tokens)

In [ ]:
%%time
df_merged_comments['parsed'] = df_merged_comments.comment.apply(parse_sentence)

In [ ]:
df = df_merged_comments.drop(['comment'], axis=1)

In [ ]:
df.reting = df.reting.apply(round)

In [ ]:
df.isnull().sum()

In [ ]:
df.head()

In [2]:
data = pd.read_csv('data_parsed.csv')

In [3]:
data.head()

,sku,categoryLevel1Id,categoryLevel2Id,brandId,reting,comment,com_len,comPos_len,comNeg_len,nCapital,nCapitalPos,nCapitalNeg,hasNegComment,hasPosComment,n_of_exc,parsed
0,20005023,401,4010201,826,2.0,"2,5 года работала и все...устала! Лампочка гор...",65.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,год работать устать лампочка гореть большой ничто
1,20020647,403,4030101,1425,2.0,Через 2 месяца после истечении гарантийного ср...,1042.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,месяц истечение гарантийный срок машинка навер...
2,20020701,401,4010401,124,4.0,пользуюсь уже три недели. нареканий ни каких н...,723.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,пользоваться неделя нарекание какой не положит...
3,30012256,203,2030301,93,5.0,Ребят этот системный блок подойдёт для игры кс...,71.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,ребята системный блок подойти игра кс го средн...
4,30011341,205,2050201,656,5.0,"я считаю, что яри замечательный телефон! Прият...",261.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12.0,считать ярь замечательный телефон приятно держ...


In [ ]:
df.reting.value_counts()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

In [ ]:
a = [[]]

In [ ]:
type(a)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['reting'], axis=1), 
                                                    df['reting'], test_size=0.33)

In [ ]:
tfidf = TfidfVectorizer(analyzer = 'word', 
                        lowercase= False, 
                        ngram_range=(1,3), 
                        min_df=5,
                        max_features = 2000).fit(X_train['parsed'])

In [ ]:
tfidf_train = tfidf.transform(X_train['parsed'])
tfidf_test = tfidf.transform(X_test['parsed'])

In [ ]:
add_train = np.array(X_train[['com_len', 'comPos_len', 'comNeg_len','brandId', 'nCapital', 'hasNegComment', 'hasPosComment', 'n_of_exc']])
add_test = np.array(X_test[['com_len', 'comPos_len', 'comNeg_len','brandId', 'nCapital', 'hasNegComment', 'hasPosComment', 'n_of_exc']])

In [ ]:
X_train_tfidf = np.hstack([tfidf_train.todense(),add_train])
X_test_tfidf = np.hstack([tfidf_test.todense(),add_test])

In [ ]:
from sklearn import preprocessing

In [ ]:
scaler = preprocessing.StandardScaler().fit(X_train_tfidf)

In [ ]:
X_train_tfidf = scaler.transform(X_train_tfidf)
X_test_tfidf = scaler.transform(X_test_tfidf)

In [ ]:
clf = SVC(C = 1.0, kernel = 'linear').fit(X_train_tfidf, np.array(y_train))

In [ ]:
clf.score(X_test_tfidf, np.array(y_test))

In [ ]:
scores = cross_val_score(clf, tfidf_features, list(newdf['reting']), cv=5)

In [ ]:
neg = df[df.reting < 2]
pos = df[df.reting > 3]

In [ ]:
list(neg['parsed'])[0].split(' ')[0]

In [ ]:
def top_words(data, n = 40): 
    al = []
    for a in data:
        for word in a.split():
            al.append(word)
    return Counter(al).most_common(n)

In [ ]:
top_words(pos['parsed'])

In [ ]:
print(top_words(neg['parsed']), '\n\n', top_words(pos['parsed']))

In [ ]:
def top_collocations(data, n = 30):
    al = []
    for a in data:
        loc = a.strip().split(' ')
        for ii in range(len(loc) - 1):
            al.append((loc[ii], loc[ii+1]))
    return Counter(al).most_common(n)

In [ ]:
top_collocations(pos['parsed'])

In [ ]:
top_collocations(neg['parsed'])

In [ ]:
top_collocations(neg['parsed'])

In [ ]:
sr = df[df.reting == 3]
top_collocations(sr['parsed'])

In [ ]:
def top_trigramms(data, n = 30):
    al = []
    dic = []
    for a in data:
        loc = a.strip().split(' ')
        for ii in range(len(loc) - 2):
            al.append((loc[ii], loc[ii+1], loc[ii+2]))
    return Counter(al).most_common(n)

In [ ]:
top_trigramms(pos['parsed'], n = len(pos['parsed']))

In [ ]:
top_trigramms(neg['parsed'])

In [ ]:
lenn = pd.Series(lenn, name = 'len')

In [ ]:
ndf = pd.concat([df, lenn], axis = 1)

In [ ]:
import seaborn as sns

In [ ]:
sns.lmplot('reting', 'len', ndf)

In [ ]:
ndf.corr()

In [ ]:
keep = {'reting': [1.0, 2.0, 3.0, 4.0, 5.0]}

newdf = df[df[list(keep)].isin(keep).all(axis=1)]

In [ ]:
newdf = newdf.dropna()

In [ ]:
tfidf = TfidfVectorizer(min_df=5, ngram_range=(1,3)).fit(newdf['parsed'])

In [ ]:
tfidf_features = tfidf.transform(newdf['parsed'])

In [ ]:
lr = LogisticRegression().fit(tfidf_features, list(newdf['reting']))

In [ ]:
clf = SVC(C = 1.0, kernel = 'linear')
scores = cross_val_score(clf, tfidf_features, list(newdf['reting']), cv=5)

In [ ]:
scores.mean()